In [1]:
import os
import random
import numpy as np
from numba import jit
from datetime import datetime

import seaborn as sns
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.patches import Circle
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation

from IPython.display import clear_output, display

In [2]:
out = '..\\out\\'
figsave_format = 'pdf'
figsave_dpi = 200

# Set axtick dimensions
major_size = 6
major_width = 1.2
minor_size = 3
minor_width = 1
mpl.rcParams['xtick.major.size'] = major_size
mpl.rcParams['xtick.major.width'] = major_width
mpl.rcParams['xtick.minor.size'] = minor_size
mpl.rcParams['xtick.minor.width'] = minor_width
mpl.rcParams['ytick.major.size'] = major_size
mpl.rcParams['ytick.major.width'] = major_width
mpl.rcParams['ytick.minor.size'] = minor_size
mpl.rcParams['ytick.minor.width'] = minor_width

# Seaborn style settings
sns.set_style({'axes.axisbelow': True,
               'axes.edgecolor': '.8',
               'axes.facecolor': 'white',
               'axes.grid': True,
               'axes.labelcolor': '.15',
               'axes.spines.bottom': True,
               'axes.spines.left': True,
               'axes.spines.right': True,
               'axes.spines.top': True,
               'figure.facecolor': 'white',
               'font.family': ['sans-serif'],
               'font.sans-serif': ['Arial',
                'DejaVu Sans',
                'Liberation Sans',
                'Bitstream Vera Sans',
                'sans-serif'],
               'grid.color': '.8',
               'grid.linestyle': '--',
               'image.cmap': 'rocket',
               'lines.solid_capstyle': 'round',
               'patch.edgecolor': 'w',
               'patch.force_edgecolor': True,
               'text.color': '.15',
               'xtick.bottom': True,
               'xtick.color': '.15',
               'xtick.direction': 'in',
               'xtick.top': True,
               'ytick.color': '.15',
               'ytick.direction': 'in',
               'ytick.left': True,
               'ytick.right': True})

## Physical Constants

In [3]:
g = 9.81

## RK4

In [4]:
@jit(nopython=True)
def derivates(X, V, i):
    assert X.size == V.size
    # Create placeholder array to store output of the RK4 step
    # f = [t, r_x, r_y, r_z, v_x, v_y, v_z]
    dXdt = np.zeros(len(X))
    dVdt = np.zeros(len(V))

    # Calculate derivates for RK4
    # ---------------------------
    # d(t)/dt = 1
    #
    # d(r_x)/dt = v_x
    # d(r_y)/dt = v_y
    # etc...
    #
    # d(v_x) = a_x
    # d(v_y)/dt = a_y
    # etc...
    # where : a_i = - G * sum_j {m_j * (r_i - r_j) / |r_i - r_j|^3}
    dXdt = V
    dVdt = - G_AU * sum_j(X, i)

    return dXdt, dVdt

In [5]:
@jit(nopython=True)
def RK4_step(coordinates, velocities, i, dt):
    X = coordinates[i]
    V = velocities[i]
    E_i = 1/2 * geometries[i,0] * np.linalg.norm(V)
    
    k1X, k1V = derivates(X, V, i)
    k1X = k1X * dt
    k1V = k1V * dt
    
    k2X, k2V = derivates(X + 0.5 * k1X, V + 0.5 * k1V, i)
    k2X = k2X * dt
    k2V = k2V * dt
    
    k3X, k3V = derivates(X + 0.5 * k2X, V + 0.5 * k2V, i)
    k3X = k3X * dt
    k3V = k3V * dt
    
    k4X, k4V = derivates(X + k3X, V + k3V, i) 
    k4X = k4X * dt
    k4V = k4V * dt

    dX = (k1X + 2 * k2X + 2 * k3X + k4X) / 6.0
    dV = (k1V + 2 * k2V + 2 * k3V + k4V) / 6.0

    return dX, dV